In [ ]:
ANR HyperOtlet

1. travail effectué sur le fichier du Traité de Documentation(1934) de Paul Otlet en txt issu de Wikisource, converti à partir du epub avec calibre. 

2. Démarche exploratoire : construire le vocabulaire vectorisé, une matrice tfidf puis tenter un clustering (en faisant varier k) 
    
3. Pour la suite différentes poursuite hypothèses LDA ? ou même démarche + stopwords et stemming 

4. Autres pistes de Text Mining : découvrir des néologismes ? représentation graphique...

In [28]:
# import des bibliothèques nécessaires
#manipulation des données regex etc...
import pandas as pd
import os, sys, email, re
import os.path
from optparse import OptionParser
import numpy as np

# Plotting et visualisation 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set_style('whitegrid')
# fail to import wordcloud ??? 

#nlp et corpus
import gensim
from gensim import corpora
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.stem.porter import PorterStemmer
from nltk.tokenize.regexp import RegexpTokenizer
from subprocess import check_output

# algos : clustering TF-IDF, LDA et PCa
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.decomposition import LatentDirichletAllocation
# études en réseau 
import networkx as nx


In [48]:
import nltk
from nltk.corpus import stopwords 



<WordListCorpusReader in '.../corpora/stopwords' (not loaded yet)>

In [44]:
# Nous allons ensuite ouvrir le txt des entrées du TD a l'aide d'une regex
with open("C://Users//henry//AnacondaProjects/HyperOtlet/TD.txt", 'r', encoding='UTF-8') as TD:
    TD = [line.strip() for line in TD.readlines()]
# vectorisation 
vectorizer = CountVectorizer( encoding='utf-8', max_df=1, stop_words='english')
vectorizer.fit(TD)
vocab = vectorizer.get_feature_names()
vv = vectorizer.vocabulary_
#mis en commentaire pour plus de lisibilité
#type(vv)
#len(vv)
#print(vv)



{'editiones': 3752, '2018': 172, 'rumeur': 9287, 'clameur': 2270, 'accumulée': 477, 'déconcertant': 3463, 'affolant': 592, 'pire': 8017, 'inondation': 5582, 'déluge': 3562, 'épandre': 11531, 'spécifiques': 10049, 'abordant': 404, 'régulatrices': 9384, 'engageant': 3930, 'efforcé': 3779, 'montrés': 6995, 'organisatrices': 7533, 'administrent': 538, 'cataloguée': 1998, 'entourée': 4003, 'productrices': 8308, 'demeurées': 3043, 'conduirait': 2519, 'linné': 6335, 'arrivés': 1070, 'promouvoir': 8356, 'déverser': 3718, 'équivaudront': 11589, 'désiderata': 3661, 'ignoré': 5298, 'biblifié': 1465, 'documentalisé': 3275, 'interinfluences': 5706, 'séparent': 10388, 'préexistait': 8445, 'logiciens': 6411, 'exalter': 4149, 'exploré': 4212, 'plongeant': 8071, 'disputant': 3214, 'communicative': 2408, 'bifurcation': 1512, 'divergentes': 3245, 'nomie': 7280, 'biblionomie': 1481, 'normatif': 7296, 'confectionne': 2524, 'accueille': 471, 'délimiter': 3554, 'analysées': 828, 'employerons': 3858, 'grapho'

In [50]:
# Formalisme sac de mots 
lines_bag_of_words = vectorizer.transform(TD).toarray()
lines_bag_of_words.shape

(19320, 11670)

In [58]:
#Affichage des mots les plus fréquents (selon TFxIDF) en gardant les majuscules et les stopwords :

def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()
    ''' D[D < min_tfidf] = 0 '''
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)

top_mean_feats(X_TD, features_TD)

tfidf_vectorizer = TfidfVectorizer(lowercase=False)
tfidf_vectorizer.fit(TD)

X_TD = tfidf_vectorizer.transform(TD)
features_TD = tfidf_vectorizer.get_feature_names()

D_tf = X_TD.toarray()
tf_sum = np.sum(D_tf, axis=0)
top_tfidf_feats(tf_sum, features_TD)

NameError: name 'top_tfidf_feats' is not defined

In [ ]:
On peut maintenant construire la matrice documents * termes : à l’aide de ce sac. Ne disposant pas de catégories nous ne pouvons pas appliquer d’apprentissage supervisé Pour déterminer quel algorithme appliquer, j’ai fait appel à cette carte.http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html Pour ce type de données, le clustering semble plus indiqué : cela va permettre d’utiliser la distance euclidienne pour grouper les mots les plus proches les uns des autres dans notre corpus.

In [60]:
clustering = KMeans(n_clusters=8, init='k-means++', max_iter=1000, n_init=1)
clustering.fit(X_TD)
#nous souhaitons affichcher les clusters avec les termes les plus proches les uns des autres dans le corpus
print("Termes les plus fréquents par clusters")
centroides = clustering.cluster_centers_.argsort()[:, ::-1]
termes = features_TD
for i in range(8):
    print("Cluster %d:" % i),
    for ind in centroides[i, :10]:
        print(' %s' % termes[ind]),
 
    
    


Termes les plus fréquents par clusters
Cluster 0:
 millions
 Post
 Britannique
 230
 servis
 140
 abonnements
 distribué
 postaux
 échantillons
Cluster 1:
 de
 des
 et
 du
 Les
 La
 en
 Le
 la
 les
Cluster 2:
 Coran
 révélations
 de
 les
 inintelligibilité
 courts
 dans
 est
 la
 ordre
Cluster 3:
 Calendars
 Britannique
 Empire
 Universités
 comprennent
 50
 près
 année
 seule
 pages
Cluster 4:
 Organisation
 mondiale
 de
 Travail
 échelon
 des
 intellectuel
 internationale
 425
 et
Cluster 5:
 trop
 pharmaciens
 drug
 Trop
 stores
 sensationnelles
 bluff
 inflation
 prévoit
 de
Cluster 6:
 de
 la
 les
 des
 et
 le
 en
 est
 un
 une
Cluster 7:
 spéculation
 guerre
 papier
 âpreté
 vertigineuses
 hausses
 normales
 subi
 trusts
 gain


les analyse par Vector Space model et le clustering par Kmeans sont inexploitables tels quels : le besoin de stopwords personnalisés et FR se fait sentir.
pour la suite Modèle probabiliste avec Allocation de Dirichlet Latente plus adaptée ? [en cours]